In [6]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import os
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from ipywidgets import fixed, Layout, Button, Box
import scipy.optimize as spopt 
import io
import kmc.tools
%matplotlib widget
#%matplotlib inline
plt.style.use(['labplot', 'labplot_note'])

###################
#Preliminary setup#
###################

#getting the path file to dump txts after
try:
    with open('pathfile.txt','r') as p:
        for line in p:
            path_bash = line
except:
    path_bash = ''
#files = [i for i in os.listdir('.') if '.txt' in i]


###Buttons to be used####
#read file button
run_but = widgets.ToggleButton(
    value=False,
    description='Read File',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

#dropdown = widgets.Dropdown(options=files, values=files[0],description='File name',disabled=False)

#file manager
dropdown = widgets.FileUpload(
    accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


#button to download the data from histogram
dump_but_hist = widgets.ToggleButton(
    value=False,
    description='Export Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    )
##########################

In [7]:
#####################################
# widgets are included through two functions:
# XXX_widget : recieves the variables from the interface's buttons and treat it to become suitable for analysis
# XXX_func   : carries out the analysis and visualization
#####################################


#### TRPL #######################
def trpl_func(dataframe,bin_num,tot_len,dump_but,legend,area_display):
    plt.figure(5)
    plt.clf()
    fluorescence = dataframe['Time'].to_numpy(float)/1000
    hist, bins = kmc.tools.trpl(fluorescence,bin_num=bin_num) 
    
    plt.xlabel("Time (ns)")          
    plt.ylabel("Intensity (a.u.)")   
    plt.ylim([0,1.05])
    
    plt.plot(bins,hist/max(hist), label=legend)
    avg_lf = np.mean(fluorescence)
    cutoff = np.max(fluorescence)
    area = 0
    for h in range(len(hist)-1):
        area += hist[h]*(bins[h+1]-bins[h])
        if area >= area_display/100:
            cutoff = bins[h]
            break

    plt.xlim([0,cutoff])
    print(f'The average lifetime is: {avg_lf:.2f} ns')
    print(f'We are using {100*len(fluorescence)/tot_len:.2f} % of the total data')
    
    plt.legend()
    plt.show()
    
    if dump_but:
        hist_data = np.column_stack([bins, hist/max(hist)])
        np.savetxt(path_bash+'/trpl.txt'     , hist_data, fmt=['%f','%f'])
    
def trpl_widget(data,tot_len,**kwargs):
    #getting the restrictions to the dataset from the buttons
    thekeys = list(set([i[:-1] for i in kwargs.keys()]))
    # filtering the dataframe to leave only the dataset of interest
    data_filtered = []
    for key in thekeys:
        mats,energy,deaths,status = [kwargs[i] for i in kwargs.keys() if key in i]
        data_filtered.append(kmc.tools.filter_per_particle(data,key,deaths,mats,energy,status))    
    data_filtered =  pd.concat(data_filtered)
    
    try:
        bin_slider = widgets.IntSlider(
        value=100,
        max=300,
        min=2,
        step=1,
        description='Bins:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',**dict(_css = (('.widget-label', 'min-width', '20ex'),),margin = '0px 0px 5px 12px')
        )
        
        area_slider = widgets.IntSlider(
        value=50,
        max=100,
        min=1,
        step=1,
        description='Fraction:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',**dict(_css = (('.widget-label', 'min-width', '20ex'),),margin = '0px 0px 5px 12px')
        )

        dump_but = widgets.ToggleButton(
            value=False,
            description='Export Data',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            icon='check'
        )

        #gettin which processes were selected to the trpl calculation
        legend = []
        for i in kwargs.keys():
            if '2' in i:
                try:
                    legend.append(kwargs[i][0])
                except:
                    pass
        legend = ', '.join(legend).capitalize() #capitalize(' '.join(legend))
        widgets.interact(trpl_func,dataframe=fixed(data_filtered),bin_num = bin_slider,tot_len=fixed(tot_len),dump_but=dump_but,legend=fixed(legend),area_display=area_slider)
        
    except:
        print('error')
        pass
###################################
######### histogram ###############
def make_hist_plot(histogram_data):
    try:
        names, counts = zip(*histogram_data)
        print(names, counts)
        x = np.arange(len(names))
        plt.figure(3)
        plt.clf()
        plt.bar(x, height=counts)
        plt.xticks(x, names)  
        plt.show()
    except:
        pass
def histogram_func(data,name_deaths,**kwargs):
    
    singlet_boxes = [kwargs["singl_box1"], kwargs["singl_box2"], 
    kwargs["singl_box3"], kwargs["singl_box4"],
    kwargs["singl_box5"], kwargs["singl_box6"]]

    triplet_boxes = [kwargs["tripl_box1"], kwargs["tripl_box2"], 
    kwargs["tripl_box3"], kwargs["tripl_box4"],
    kwargs["tripl_box5"], kwargs["tripl_box6"]]

    hole_boxes = [kwargs["hole_box1"], kwargs["hole_box2"], 
    kwargs["hole_box3"], kwargs["hole_box4"],
    kwargs["hole_box5"], kwargs["hole_box6"]]
    
    electron_boxes = [kwargs["elect_box1"], kwargs["elect_box2"], 
    kwargs["elect_box3"], kwargs["elect_box4"],
    kwargs["elect_box5"], kwargs["elect_box6"]]
    
    dump_but = kwargs["dump_but"]
    
    par_boxes = {}
    par_boxes['singlet']  = singlet_boxes
    par_boxes['triplet']  = triplet_boxes
    par_boxes['electron'] = electron_boxes
    par_boxes['hole']     = singlet_boxes
    
    histogram_data = []
    
    #looping over the checked boxes, getting the filtered data and plotting
    for part in name_deaths:
        list_death = name_deaths[part]
        for i in range(len(list_death)):
            death = list_death[i]
            it_is_on = par_boxes[part][i]
            #print(death,it_is_on)
            if it_is_on: # if histogram's interface check box is pressed
                data_part = data[part]
                data_filter = kmc.tools.filter_data_list(data_part,'CausaMortis',[death])
                n_filter = len(data_filter)
                histogram_data.append([part+'_'+death,n_filter])
    if histogram_data:
        make_hist_plot(histogram_data)
        if dump_but:
            hist_data = np.array(histogram_data,dtype='str')
            np.savetxt(path_bash+'/histogram.txt' , hist_data, delimiter=' ', fmt='%s')
            
def histogram_widget(data,tot_len,**kwargs):
    display(data)
    #filtering using the common filtering interface
    #particles_filter = {}
    #particles_filter["singlet"] = [kwargs["singl_mats"], kwargs["singl_energy"], 
    #kwargs["singl_deaths"], kwargs["singl_status"]]
    #
    #particles_filter["triplet"] = [kwargs["tripl_mats"], kwargs["tripl_energy"], 
    #kwargs["tripl_deaths"], kwargs["tripl_status"]]
    #
    #particles_filter["hole"]    = [kwargs["hole_mats"], kwargs["hole_energy"], 
    #kwargs["hole_deaths"], kwargs["hole_status"]]
    #
    #particles_filter["electron"] = [kwargs["elect_mats"], kwargs["elect_energy"], 
    #kwargs["elect_deaths"], kwargs["elect_status"]]

    #data_filtred = {}
    #for particle in particles_filter:
    #    mats,energy,deaths,status = particles_filter[particle]
    #    data_filtred[particle] = kmc.tools.filter_per_particle(data,particle,deaths,mats,energy,status)  
    
    #getting the death processes relevant for each particle
    #death_list = {}
    #death_list["singlet"]  = kwargs["singl_deaths"]
    #death_list["triplet"]  = kwargs["tripl_deaths"]
    #death_list["electron"] = kwargs["elect_deaths"]
    #death_list["hole"]     = kwargs["hole_deaths"]
    #
    #name_deaths = {}
    #box_singlet,deaths_singlet   = kmc.tools.get_boxes_for_a_particle(6,death_list["singlet"])
    #box_triplet,deaths_triplet   = kmc.tools.get_boxes_for_a_particle(6,death_list["triplet"])
    #box_hole,deaths_hole         = kmc.tools.get_boxes_for_a_particle(6,death_list["hole"])
    #box_electron,deaths_electron = kmc.tools.get_boxes_for_a_particle(6,death_list["electron"])

    #name_deaths['singlet']  = deaths_singlet
    #name_deaths['triplet']  = deaths_triplet
    #name_deaths['hole']     = deaths_hole
    #name_deaths['electron'] = deaths_electron
    
    #initializating the histogram widget by giving the boxes as input
    #w_hi = widgets.interactive(histogram_func,data=fixed(data_filtred),name_deaths=fixed(name_deaths),
    #                       singl_box1=box_singlet[0],singl_box2=box_singlet[1],
    #                       singl_box3=box_singlet[2],singl_box4=box_singlet[3],
    #                       singl_box5=box_singlet[4],singl_box6=box_singlet[5],
    #                       tripl_box1=box_triplet[0],tripl_box2=box_triplet[1],
    #                       tripl_box3=box_triplet[2],tripl_box4=box_triplet[3],
    #                       tripl_box5=box_triplet[4],tripl_box6=box_triplet[5],
    #                       hole_box1=box_hole[0],hole_box2=box_hole[1],
    #                       hole_box3=box_hole[2],hole_box4=box_hole[3],
    #                       hole_box5=box_hole[4],hole_box6=box_hole[5],  
    #                       elect_box1=box_electron[0],elect_box2=box_electron[1],
    #                       elect_box3=box_electron[2],elect_box4=box_electron[3],
    #                       elect_box5=box_electron[4],elect_box6=box_electron[5],
    #                       dump_but = dump_but_hist)    

    #organize the boxes by particle type
    def organize_boxes(num_ele,widget_int):
        labels = ['Singlet','Triplet', 'Hole', 'Electron']
        n_boxes = int(len(widget_int.children)/num_ele)
        #print(n_boxes)
        v = widgets.VBox([widgets.Label('')])
        for k in range(n_boxes):
            rang = range(num_ele*k,(num_ele)*(k+1)-1)
            print(labels[k],rang)
            h = widgets.HBox([widget_int.children[i] for i in rang])
            v = widgets.VBox([v,widgets.Label(labels[k]),h])
        v = widgets.VBox([v,widget_int.children[-1]])
        return v
                         
    #v = organize_boxes(6,w_hi) #organizing the pack of 6 check box, for each particle, to display              
    #display(v,dump_but_hist)   #displaying the histogram widgets
    #display(w_hi)
###################################

def data_widget(data,**kwargs):
    #getting the restrictions to the dataset from the buttons
    thekeys = list(set([i[:-1] for i in kwargs.keys()]))
    # filtering the dataframe to leave only the dataset of interest
    data_filtered = []
    for key in thekeys:
        mats,energy,deaths,status = [kwargs[i] for i in kwargs.keys() if key in i]
        data_filtered.append(kmc.tools.filter_per_particle(data,key,deaths,mats,energy,status))    
    data_filtered =  pd.concat(data_filtered)

    display(data_filtered)    

#core function
def main(file_name,run_button):
    #print('the file '+ list(file_name.keys())[0] + 'was loaded!')
    if run_button: #if 'Read file' is pressed
        name = list(file_name.keys())[0]
        data = file_name[name]['content']
        data = io.StringIO(data.decode('utf-8'))
        data = pd.read_csv(data)#, delim_whitespace=True)
        data = data[data.Time != 'END'] #removes the end of round lines
        
        tot_len    = np.shape(data)[0]
        species    = data['Type'].unique()
        max_energy = np.amax(data['Energy'])
        min_energy = np.amin(data['Energy'])
        
        ########################################    
        #defining widgets after pre load process
        #for each particle there are two sets of widgets: 
        # 1) the ones attached to the table : [energy , mats,   deaths, status] 
        # 2) the ones attached to the widget.interactive : [energy2, mats2, deaths2, status2] 
        # Comunication between them is mediated by the report_ie_change function
        
        obserb_dics, obstab, names,boxes = [], [], [], [] 
        for part in species:
            data_part     = kmc.tools.filter_data_list(data,'Type',[part])
            unique_mats   = data_part['Location'].unique()
            unique_death  = data_part['CausaMortis'].unique()
            unique_status = data_part['Status'].unique()
            wids          = kmc.tools.get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death,unique_status)
            wids2         = kmc.tools.get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death,unique_status)
            names.extend([part+str(i) for i in range(len(wids))])
            obserb_dics.extend(wids)
            obstab.extend(wids2)
            boxes.append(kmc.tools.make_boxes(wids2))
            
        
        kw,avatar = {}, {}
        for i in range(len(names)):
            kw[names[i]]     = obserb_dics[i]
            avatar[names[i]] = obstab[i]
        
        
        ###########################
        #initializing table
        output       = widgets.Output()
        tab          = widgets.Tab()
        tab.children = boxes
        #setting up the titles of the table
        for i in range(len(species)):
            tab.set_title(i,species[i].upper())
        display(tab,output)
        
        w_data = widgets.interactive(data_widget,data=fixed(data),**kw)

        #Calling Histogram INTERACTIVE INTERFACE
        w_hist = widgets.interactive(histogram_widget,data=fixed(data),tot_len=fixed(tot_len),**kw)

        
        #Calling TRPL INTERACTIVE INTERFACE
        w_trpl = widgets.interactive(trpl_widget,data=fixed(data),tot_len=fixed(tot_len),**kw)
        
        

        accordion = widgets.Accordion(children=[w_data,w_trpl,w_hist], selected_index=None)
        accordion.set_title(0, 'DATA')
        accordion.set_title(1, 'HISTOGRAM')
        accordion.set_title(2, 'TIME RESOLVED PHOTOLUMINESCENCE')
        accordion.set_title(3, 'HEATMAP')
        display(accordion)
        
        #this function updates the atribute value upon changes in the widget. Must remain defined inside of main()
        def report_ie_change(change):#,accordion,attrib,indx):
            with output:    
                index = list(avatar.values()).index(change.owner)        
                list(kw.values())[index].value = change.new

        for wid in avatar.values():
            wid.observe(report_ie_change,names=['value'])
        
        #needed because by default widget.interactive pops up the widgets. Tuning them invisible here..
        for indx in range(len(w_trpl.children)-1):
            w_trpl.children[indx].close()
        
        
    else:
        pass 

In [8]:
###################################################
#Initializing main function and displaying widgets
###################################################
display(widgets.HTML(value = r'<p style="font-size:24px"><b>DASHBOARD KMC</b></p>'))
i = widgets.interactive(main, file_name=dropdown,run_button=run_but);
h = widgets.HBox([i.children[0],i.children[1]])
v = widgets.VBox([h,i.children[2]])
display(v)

HTML(value='<p style="font-size:24px"><b>DASHBOARD KMC</b></p>')